##### 参考
##### Alternating Direction Method of Multipliers (ADMM) で Lasso 回帰
##### https://qiita.com/s-capybara/items/48e2829ae7c1d402bff8

In [ ]:
import numpy as np

In [90]:
# ランダムシードによる乱数生成
prng = np.random.RandomState(0)

In [106]:
# Toy Dataset I
def create_dataset_1(n=100):
    x = 3 * (prng.randn(n,2) - 0.5)
    radius = [i**2 + j**2 for i,j in x]
    y = [(r > 0.7 + 0.1 * prng.randn()) and (r < 2.2 + 0.1 * prng.randn()) for r in radius]
    y = np.array([2 * t_y - 1 for t_y in y])
    return x,y

# Toy Dataset II
def create_dataset_2(n=40):
    omega = prng.randn()
    noise = 0.8 * prng.randn(1,n)
    x = prng.randn(n,2)
    y = 2 * (omega * x[:,0]+x[:,1] + noise > 0) -1
    y = y.reshape(n)
    return x,y

In [88]:
n=100

x,y = create_dataset_1(n)

mean = np.mean(x, axis=0)
std = np.std(x, axis=0)

A = x
b = y

model = Admm(lambd=1.0, rho=1.0, max_iter=1000)
model.fit(A, b)
print("Lasso by ADMM")
print(model.coef_)

Lasso by ADMM
[ 0.0172029227  0.0542859795]


In [86]:
# テストデータセットを用意
x,t = create_dataset_1(n)

y = model.predict(np.array(x))
y = np.where(y > 0,1,-1)
acc = np.where(t == y,1,0)
acc.sum()

72

In [137]:
# データセット2でやってみる
print("データセット2")
x,y = create_dataset_2(40)
print("x",x.shape)
print("y",y.shape)

A = x
b = y

model = Admm(lambd=1.0, rho=1.0, max_iter=25)
model.fit(A, b)
print("Lasso by ADMM")
print(model.coef_)

# テストデータセットを用意
x,t = create_dataset_2(40)

y = model.predict(np.array(x))
y = np.where(y > 0,1,-1)
acc = np.where(t == y,1,0)
print("正答率",acc.sum() / len(acc))

データセット2
x (40, 2)
y (40,)
Lasso by ADMM
[ 0.0000000001 -0.0000000000]
正答率 0.425


In [5]:
#class Admm(BaseEstimator, RegressorMixin):
class Admm:
    def __init__(self, lambd=1.0, rho=1.0, max_iter=1000):
        self.lambd = lambd
        self.rho = rho
        self.threshold = lambd / rho
        self.max_iter = max_iter
        self.coef_ = None
        self.intercept_ = 0.0

    def _soft_threshold(self, x):
        t = self.threshold
        
        positive_indexes = x >= t
        negative_indexes = x <= t
        zero_indexes = abs(x) <= t
        
        y = np.zeros(x.shape)    
        y[positive_indexes] = x[positive_indexes] - t
        y[negative_indexes] = x[negative_indexes] + t
        y[zero_indexes] = 0.0
    
        return y

    def fit(self, A, b):
        N = A.shape[0]
        M = A.shape[1]
        inv_matrix = np.linalg.inv(np.dot(A.T, A) / N + self.rho * np.identity(M))
        
        x = np.dot(A.T, b) / N
        z = x.copy()
        y = np.zeros(len(x))
    
        for iteration in range(self.max_iter):
            x = np.dot(inv_matrix, np.dot(A.T, b) / N + self.rho * z - y)
            z = self._soft_threshold(x + y / self.rho)
            y += self.rho * (x - z)

        self.coef_ = x
                      
        return self

    def predict(self, X):
        y = np.dot(X, self.coef_)
        return y